Tải dữ liệu lên 

In [1]:
import pandas as pd

ames_housing = pd.read_csv('../datasets/house_prices.csv', na_values='?')

target_name = 'SalePrice'
data, target = (
    ames_housing.drop(columns=[target_name]),
    ames_housing[target_name]
)
target = (target > 200_000).astype(int)


Kiểm tra vài dòng đầu của DataFrame

In [2]:
ames_housing

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


Chọn một số đặc trưng đơn giản để thực hiện 

In [3]:
numeric_features = ['LotArea', 'FullBath', 'HalfBath']
categorical_features = ['Neighborhood', 'HouseStyle']
data = data[numeric_features + categorical_features]

Create Pipeline 

In [4]:
from sklearn.pipeline import Pipeline 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

Áp dụng biến đổi với `ColumnTransformer`

In [6]:
from sklearn.compose import ColumnTransformer 

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

Kết hợp tất cả các bước với mô hình phân loại 

In [8]:
from sklearn.linear_model import LogisticRegression 

model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression()),
    ]
)

In [9]:
# hien thi model
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['LotArea', 'FullBath',
                                                   'HalfBath']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Neighborhood',
                                                   'HouseStyle'])])),
                ('classifier', LogisticRegression())])

Đánh giá mô hình

In [10]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv=5)
scores = cv_results['test_score']
print(
    'The mean cross-validation score is: '
    f"{scores.mean():.3f} +/- {scores.std():.3f}"
)

The mean cross-validation score is: 0.859 +/- 0.018


### Ghi chú
- Trong ví dụ này, khoảng 86% số lần mô hình dự đoán đúng liệu giá nhà có vượt mức 200.000 đô hay không
- Tuy nhiên, tập đặc trưng được chọn một cách tùy ý để đơn giản bài học, không phải là phương pháp chọn đặc trưng tốt nhất
- Việc biến một bài toán dự đoán nhà (regression) thành bài toàn phân loại nhị phân với ngưỡng là 200,000 đô quá đơn giản để áp dụng trong thực thế 
- Chúng ta nên coi đây là bài toán hồi quy (regression) thì sẽ hợp lý hơn, và chúng ta sẽ làm điều đó ở phần sau khóa học MOOC